In [1]:
import glob
import warnings
import tarfile
import cv2
import keras
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, LeakyReLU
from keras.layers import BatchNormalization, Activation, Conv2D
from keras.layers import GlobalAveragePooling2D, Lambda
from keras.optimizers import Adam, RMSprop
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import to_categorical, Sequence
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from keras.models import load_model, model_from_json
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from PIL import Image
from cv2 import resize
import tensorflow as tf
from keras.applications import ResNet50
from keras import regularizers
import requests
import threading
import random
import time
import urllib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils import multi_gpu_model
from collections import Counter

%matplotlib inline

print('Keras version:', keras.__version__)
# print(os.listdir('SageMaker'))

warnings.simplefilter('default')
!pwd


Using TensorFlow backend.


Keras version: 2.2.4
/home/ec2-user/SageMaker


## Okay so in this notebook i'm going to put together what i believe will be the 'nonlandmark/islandmark' features

## Tbh i should be doing this after i get a score above 0 but fuck it


In [2]:
os.chdir("./Keras-VGG16-places365/")
from vgg16_places_365 import VGG16_Places365
os.chdir("..")

In [34]:
# n_cat = 203094
# batch_size = 64
# batch_size_predict = 128
# input_shape = (128,128)

train_path = './landmarks/train/train/'
test_path = './landmarks/test/test/'
train_images = glob.glob(train_path+'*.jpg')
test_images = glob.glob(test_path+'*.jpg')
print(len(train_images))
print(len(test_images))
sample_submission = pd.read_csv('./landmarks/recognition_sample_submission.csv')
sample_submission.shape

4130318
112821


(117703, 2)

In [35]:
train_image_ids = [image_file.replace(
    '.jpg', '').replace(train_path, '') for image_file in train_images]

train_df = pd.DataFrame(index=list(range(0,len(train_image_ids))))
train_df['filename'] = pd.Series(train_images, index=list(range(0,len(train_image_ids))))
train_df['ids'] = train_image_ids
test_image_ids = [image_file.replace(
    '.jpg', '').replace(test_path, '') for image_file in test_images]
test_df = pd.DataFrame(index=list(range(0,len(test_image_ids))))
test_df['filename'] = pd.Series(test_images, index=list(range(0,len(test_image_ids))))
test_df['ids'] = test_image_ids

## going to create a cell for all the functions that I got fromthe 19th place solution preprocessing steps:


In [5]:
def check_size(url):
    r = requests.get(url, stream=True)
    return int(r.headers['Content-Length'])

def download_file(url, filename, bar=True):
    """
    Helper method handling downloading large files from `url` to `filename`. Returns a pointer to `filename`.
    """
    try:
        chunkSize = 1024
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as f:
            if bar:
                pbar = tqdm( unit="B", total=int( r.headers['Content-Length'] ) )
            for chunk in r.iter_content(chunk_size=chunkSize): 
                if chunk: # filter out keep-alive new chunks
                    if bar: 
                        pbar.update (len(chunk))
                    f.write(chunk)
        return filename
    except Exception as e:
        print(e)
        return
    
def download_image_cv2_urllib(url):
    """
    Modifying the url to download the 360p or 720p version actually slows it down. 
    """
    try:
        resp = urllib.request.urlopen(url)
        foo = np.asarray(bytearray(resp.read()), dtype="uint8")
        foo = cv2.imdecode(foo, cv2.IMREAD_COLOR)
        foo = cv2.resize(foo,(128, 128), interpolation=cv2.INTER_AREA)
        return foo
    except:
        return np.array([])


In [36]:
train = pd.read_csv("train.csv",index_col='id')
print(train.head())
print(train.shape)
print("Number of classes {}".format(len(train.landmark_id.unique())))

NUM_THRESHOLD = 420

counts = dict(Counter(train['landmark_id']))
landmarks_dict = {x:[] for x in train.landmark_id.unique() if counts[x] >= NUM_THRESHOLD}
NUM_CLASSES = len(landmarks_dict)
print("Total number of valid classes: {}".format(NUM_CLASSES))

i = 0
landmark_to_idx = {}
idx_to_landmark = []
for k in landmarks_dict:
    landmark_to_idx[k] = i
    idx_to_landmark.append(k)
    i += 1
    
train['filename'] = pd.Series(train_images, index=train_image_ids)

all_urls = train['url'].tolist()
all_landmarks = train['landmark_id'].tolist()
valid_urls_dict = {x[0].split("/")[-1]:landmark_to_idx[x[1]] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict}
valid_urls_list = [x[0] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict]

NUM_EXAMPLES = len(valid_urls_list)
print("Total number of valid examples: {}".format(NUM_EXAMPLES))

                                                                url  \
id                                                                    
6e158a47eb2ca3f6  https://upload.wikimedia.org/wikipedia/commons...   
202cd79556f30760  http://upload.wikimedia.org/wikipedia/commons/...   
3ad87684c99c06e1  http://upload.wikimedia.org/wikipedia/commons/...   
e7f70e9c61e66af3  https://upload.wikimedia.org/wikipedia/commons...   
4072182eddd0100e  https://upload.wikimedia.org/wikipedia/commons...   

                  landmark_id  
id                             
6e158a47eb2ca3f6       142820  
202cd79556f30760       104169  
3ad87684c99c06e1        37914  
e7f70e9c61e66af3       102140  
4072182eddd0100e         2474  
(4132914, 2)
Number of classes 203094
Total number of valid classes: 339
Total number of valid examples: 252356


In [7]:
test_info_full = pd.read_csv('test.csv', index_col='id')
test_info_full.head()

test_info = test_info_full.loc[test_image_ids]
test_info['filename'] = pd.Series(test_images, index=test_image_ids)

test_info.head()

,url,filename
id,,
fa8d5a81a16f2f2f,https://lh3.googleusercontent.com/-_SAJTBt3Y64...,./landmarks/test/test/fa8d5a81a16f2f2f.jpg
b81a0a45f9b1ee97,https://lh3.googleusercontent.com/-9sFSIOCzIOs...,./landmarks/test/test/b81a0a45f9b1ee97.jpg
570e28cc63fab858,https://lh3.googleusercontent.com/-7Eld7yUfAB0...,./landmarks/test/test/570e28cc63fab858.jpg
b8bc63608b5fef1a,https://lh3.googleusercontent.com/-JdgzGjeS9NE...,./landmarks/test/test/b8bc63608b5fef1a.jpg
54cfd1f5f683b966,https://lh3.googleusercontent.com/-krCM7YZ3FpU...,./landmarks/test/test/54cfd1f5f683b966.jpg


In [8]:
n_cat = 203094 #number of unique classes (yikes)
input_shape = (128,128)

In [9]:
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse=True, n_values=n_cat)

train['label'] = label_encoder.fit_transform(train['landmark_id'])
train['one_hot'] = one_hot_encoder.fit_transform(
                    train['label'].values.reshape(-1, 1))

In [10]:
def load_images(info, input_shape = input_shape):
    input_shape = tuple(input_shape)
    imgs = np.zeros((len(info), input_shape[0], input_shape[1], 3))

    for i in range(len(info)):
        fname = info.iloc[i]['filename']
        try:
            img = cv2.cvtColor(
                  cv2.resize(cv2.imread(fname),input_shape),
                  cv2.COLOR_BGR2RGB)
        except:
            warnings.warn('Warning: could not read image: '+ fname +
                          '. Use black img instead.')
            img = np.zeros((input_shape[0], input_shape[1], 3))
        imgs[i,:,:,:] = img
    
    return imgs
def load_cropped_images(info, crop_p=0.2, crop='random'):
    new_res = np.array([int(input_shape[0]*(1+crop_p)), int(input_shape[1]*(1+crop_p))])
    if crop == 'random':
        cx0 = np.random.randint(new_res[0] - input_shape[0], size=len(info))
        cy0 = np.random.randint(new_res[1] - input_shape[1], size=len(info))
    else:
        if crop == 'central':
            cx0, cy0 = (new_res - input_shape) // 2                
        if crop == 'upper left':
            cx0, cy0 = 0, 0
        if crop == 'upper right':
            cx0, cy0 = new_res[1] - input_shape[1], 0
        if crop == 'lower left':
            cx0, cy0 = 0, new_res[0] - input_shape[0]
        if crop=='lower right':
            cx0, cy0 = new_res - input_shape        
        cx0 = np.repeat(np.expand_dims(cx0, 0), len(info))
        cy0 = np.repeat(np.expand_dims(cy0, 0), len(info))

    cx1 = cx0 + input_shape[0]
    cy1 = cy0 + input_shape[1]
    
    raw_imgs = load_images(info, input_shape=tuple(new_res))
    
    cropped_imgs = np.zeros((len(info), input_shape[0], input_shape[1], 3))
    for ind in range(len(info)):
        cropped_imgs[ind,:,:,:] = raw_imgs[ind,
                                           cy0[ind]:cy1[ind],
                                           cx0[ind]:cx1[ind], :]
    
    return cropped_imgs


def get_image_gen(info_arg, 
                  shuffle=True, 
                  image_aug=True, 
                  eq_dist=False, 
                  n_ref_imgs=16, 
                  crop_prob=0.5, 
                  crop_p=0.5):
    if image_aug:
        datagen = ImageDataGenerator(
            rotation_range=4.,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.5,
            channel_shift_range=25,
            horizontal_flip=True,
            fill_mode='nearest')
        
        if crop_prob > 0:
            datagen_crop = ImageDataGenerator(
                rotation_range=4.,
                shear_range=0.2,
                zoom_range=0.1,
                channel_shift_range=20,
                horizontal_flip=True,
                fill_mode='nearest')
        
    count = len(info_arg)
    while True:
        if eq_dist:
            def sample(df):
                return df.sample(min(n_ref_imgs, len(df)))
            info = info_arg.groupby('landmark_id', group_keys=False).apply(sample)
        else:
            info = info_arg
        print('Generate', len(info), 'for the next round.')
        
        #shuffle data
        if shuffle and count >= len(info):
            info = info.sample(frac=1)
            count = 0
            
        # load images
        for ind in range(0,len(info), batch_size):
            count += batch_size

            y = info['landmark_id'].values[ind:(ind+batch_size)]
            
            if np.random.rand() < crop_prob:
                imgs = load_cropped_images(info.iloc[ind:(ind+batch_size)], 
                                           crop_p=crop_p*np.random.rand() + 0.01, 
                                           crop='random')
                if image_aug:
                    cflow = datagen_crop.flow(imgs, 
                                              y, 
                                              batch_size=imgs.shape[0], 
                                              shuffle=False)
                    imgs, y = next(cflow)                    
            else:
                imgs = load_images(info.iloc[ind:(ind+batch_size)])
                if image_aug:
                    cflow = datagen.flow(imgs, 
                                       y, 
                                       batch_size=imgs.shape[0], 
                                       shuffle=False)
                    imgs, y = next(cflow)             

            imgs = preprocess_input(imgs)
    
            y_l = label_encoder.transform(y[y>=0.])        
            y_oh = np.zeros((len(y), n_cat))
            y_oh[y >= 0., :] = one_hot_encoder.transform(y_l.reshape(-1,1)).todense()
                    
            yield imgs, y_oh
            
train_gen = get_image_gen(train, 
                          eq_dist=False, 
                          n_ref_imgs=256, 
                          crop_prob=0.5, 
                          crop_p=0.5)

In [11]:
n_cat_train=train['landmark_id'].nunique()
print(n_cat_train)
if n_cat_train != n_cat:
    warnings.warn('Warning: The training data is not compatible.')

203094


In [12]:
landmarks = pd.read_csv('topn_all_info.csv')

In [25]:
landmarks['landmark_file'] = [image_file.replace(
    './landmarks/train/train/', '').replace(image_file, '') for image_file in landmarks['filename']]

In [27]:
landmarks['landmark_id'] = [image_file.replace(
    '.jpg', '').replace(image_file, '') for image_file in landmarks['landmark_file']]

In [28]:
landmarks.head()

,filename,p0,p1,p2,p0_label,p0_landmark,p1_label,p1_landmark,p2_label,p2_landmark,landmark_id,landmark_file
0,./landmarks/train/train/687c09f942938f4e.jpg,236,14,240,/m/museum/indoor,non-landmark,/a/archive,non-landmark,/n/nursery,non-landmark,687c09f942938f4e,687c09f942938f4e.jpg
1,./landmarks/train/train/11704d4b86f8fe1a.jpg,309,76,234,/s/snowfield,landmark,/c/campsite,landmark,/m/mountain_snowy,landmark,11704d4b86f8fe1a,11704d4b86f8fe1a.jpg
2,./landmarks/train/train/67e34bedf25bd3d2.jpg,93,236,240,/c/clean_room,non-landmark,/m/museum/indoor,non-landmark,/n/nursery,non-landmark,67e34bedf25bd3d2,67e34bedf25bd3d2.jpg
3,./landmarks/train/train/05250fb79c967abb.jpg,190,187,186,/i/iceberg,landmark,/i/ice_shelf,landmark,/i/ice_floe,landmark,05250fb79c967abb,05250fb79c967abb.jpg
4,./landmarks/train/train/ceb9fde5122403d6.jpg,296,183,107,/s/schoolhouse,landmark,/h/house,landmark,/c/cottage,landmark,ceb9fde5122403d6,ceb9fde5122403d6.jpg


In [30]:
landmarks.set_index('landmark_id',inplace=True)

In [37]:
# train.set_index('landmark_id',inplace=True)

In [41]:
train.head()

,url,landmark_id,filename
id,,,
6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820,./landmarks/train/train/6e158a47eb2ca3f6.jpg
202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169,./landmarks/train/train/202cd79556f30760.jpg
3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914,./landmarks/train/train/3ad87684c99c06e1.jpg
e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140,./landmarks/train/train/e7f70e9c61e66af3.jpg
4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474,./landmarks/train/train/4072182eddd0100e.jpg


In [43]:
index = landmarks.index
landmark_ids = []
for ids in index:
    if ids in train.index:
        landmark_ids.append(train['landmark_id'].loc[ids])

In [46]:
landmarks['landmark_id'] = landmark_ids

In [47]:
landmarks.head()

,filename,p0,p1,p2,p0_label,p0_landmark,p1_label,p1_landmark,p2_label,p2_landmark,landmark_file,landmark_id
landmark_id,,,,,,,,,,,,
687c09f942938f4e,./landmarks/train/train/687c09f942938f4e.jpg,236,14,240,/m/museum/indoor,non-landmark,/a/archive,non-landmark,/n/nursery,non-landmark,687c09f942938f4e.jpg,198059
11704d4b86f8fe1a,./landmarks/train/train/11704d4b86f8fe1a.jpg,309,76,234,/s/snowfield,landmark,/c/campsite,landmark,/m/mountain_snowy,landmark,11704d4b86f8fe1a.jpg,136542
67e34bedf25bd3d2,./landmarks/train/train/67e34bedf25bd3d2.jpg,93,236,240,/c/clean_room,non-landmark,/m/museum/indoor,non-landmark,/n/nursery,non-landmark,67e34bedf25bd3d2.jpg,180256
05250fb79c967abb,./landmarks/train/train/05250fb79c967abb.jpg,190,187,186,/i/iceberg,landmark,/i/ice_shelf,landmark,/i/ice_floe,landmark,05250fb79c967abb.jpg,151989
ceb9fde5122403d6,./landmarks/train/train/ceb9fde5122403d6.jpg,296,183,107,/s/schoolhouse,landmark,/h/house,landmark,/c/cottage,landmark,ceb9fde5122403d6.jpg,151069


In [48]:
landmarks.to_csv('topn_all_info.csv',index=False)

In [67]:
# subset 15% of the training set
train_sample = train.sample(n=61993,random_state=42)
train_sample.shape

(61993, 5)

In [68]:
train_sample.isna().sum()

url             0
landmark_id     0
filename       52
label           0
one_hot         0
dtype: int64

In [69]:
train_sample = train_sample.dropna(axis=0)
train_sample.isna().sum()

url            0
landmark_id    0
filename       0
label          0
one_hot        0
dtype: int64

In [84]:
train_sample.head()

,url,landmark_id,filename,label,one_hot
id,,,,,
687c09f942938f4e,https://upload.wikimedia.org/wikipedia/commons...,198059,./landmarks/train/train/687c09f942938f4e.jpg,198059,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
11704d4b86f8fe1a,https://upload.wikimedia.org/wikipedia/commons...,136542,./landmarks/train/train/11704d4b86f8fe1a.jpg,136542,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
67e34bedf25bd3d2,http://upload.wikimedia.org/wikipedia/commons/...,180256,./landmarks/train/train/67e34bedf25bd3d2.jpg,180256,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
05250fb79c967abb,https://upload.wikimedia.org/wikipedia/commons...,151989,./landmarks/train/train/05250fb79c967abb.jpg,151989,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
ceb9fde5122403d6,https://upload.wikimedia.org/wikipedia/commons...,151069,./landmarks/train/train/ceb9fde5122403d6.jpg,151069,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."


In [70]:
type(train_sample['filename'][2])

str

In [89]:
from cv2 import resize

images_for_train = train_sample['filename'].values
final_filenames = []
all_images_resized = []

for filename in images_for_train: 
#     print(type(filename))
    f = Image.open(filename)
    im = np.array(f.resize((224, 224), Image.LANCZOS))    
    if len(im.shape) == 3:
        all_images_resized.append(im)
        final_filenames.append(filename)
        

In [93]:
images_df = pd.DataFrame(final_filenames,columns=['filenames'])
images_df['all_images_resized'] = all_images_resized

In [94]:
len(images_df)

61614

In [75]:
test = np.expand_dims(all_images_resized[2], 0)
test.shape

(1, 224, 224, 3)

In [86]:
len(all_images_resized)

61614

In [99]:
# all_images = []
# for i in range(0,len(all_images_resized)):
#     x = all_images_resized[i].shape
#     if len(x) == 3:
#         all_images.append(all_images_resized[i])
#     else:
#         pass

In [83]:
len(all_images)

61614

In [87]:
p0, p1, p2 = [], [], []

# Places365 Model
model = VGG16_Places365(weights='places')
topn = 5

# Loop through all images
for image in all_images:
    
    # Predict Top N Image Classes
    expand_image = np.expand_dims(image, 0)
    topn_preds = np.argsort(model.predict(expand_image)[0])[::-1][0:topn]
    p0.append(topn_preds[0])
    p1.append(topn_preds[1])
    p2.append(topn_preds[2])

ValueError: Length of values does not match length of index

In [97]:
# Create dataframe for later usage
topn_df = pd.DataFrame()
topn_df['filename'] = np.array(images_df['filenames'])
topn_df['p0'] = np.array(p0)
topn_df['p1'] = np.array(p1)
topn_df['p2'] = np.array(p2)
topn_df.to_csv('topn_class_numbers.csv', index = False)

# Summary
topn_df.head()

,filename,p0,p1,p2
0,./landmarks/train/train/687c09f942938f4e.jpg,236,14,240
1,./landmarks/train/train/11704d4b86f8fe1a.jpg,309,76,234
2,./landmarks/train/train/67e34bedf25bd3d2.jpg,93,236,240
3,./landmarks/train/train/05250fb79c967abb.jpg,190,187,186
4,./landmarks/train/train/ceb9fde5122403d6.jpg,296,183,107


In [98]:
class_information = pd.read_csv('./Keras-VGG16-places365/categories_places365_extended.csv')
print(class_information.head())

# Set Class Labels
for col in ['p0', 'p1', 'p2']:
    topn_df[col + '_label'] = topn_df[col].map(class_information.set_index('class')['label'])
    topn_df[col + '_landmark'] = topn_df[col].map(class_information.set_index('class')['io'].replace({1:'non-landmark', 2:'landmark'}))
topn_df.to_csv('topn_all_info.csv', index = False)

# Summary
topn_df.head() 

                 label  class  io
0          /a/airfield      0   2
1    /a/airplane_cabin      1   1
2  /a/airport_terminal      2   1
3            /a/alcove      3   1
4             /a/alley      4   2


,filename,p0,p1,p2,p0_label,p0_landmark,p1_label,p1_landmark,p2_label,p2_landmark
0,./landmarks/train/train/687c09f942938f4e.jpg,236,14,240,/m/museum/indoor,non-landmark,/a/archive,non-landmark,/n/nursery,non-landmark
1,./landmarks/train/train/11704d4b86f8fe1a.jpg,309,76,234,/s/snowfield,landmark,/c/campsite,landmark,/m/mountain_snowy,landmark
2,./landmarks/train/train/67e34bedf25bd3d2.jpg,93,236,240,/c/clean_room,non-landmark,/m/museum/indoor,non-landmark,/n/nursery,non-landmark
3,./landmarks/train/train/05250fb79c967abb.jpg,190,187,186,/i/iceberg,landmark,/i/ice_shelf,landmark,/i/ice_floe,landmark
4,./landmarks/train/train/ceb9fde5122403d6.jpg,296,183,107,/s/schoolhouse,landmark,/h/house,landmark,/c/cottage,landmark
